In [2]:
import csv, pickle

# создаем модули для работы с файлами разных форматов:

def load_csv(file_path):          #загрузка данных из csv файла
    try:
        with open(file_path, 'r', newline='') as file:
            reader = csv.reader(file)
            table = [row for row in reader]
        return table
    except FileNotFoundError:
        raise FileNotFoundError("File not found")
        
def load_pickle(file_path):       #загрузка данных из pickle файла
    try:
        with open(file_path, 'rb') as file:
            table = pickle.load(file)
        return table
    except FileNotFoundError:
        raise FileNotFoundError("File not found")

def save_csv(table, file_path):     #сохранние данных в виде csv файла
    try:
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(table)
    except Exception as e:
        raise e

def save_pickle(table, file_path):  #сохранние данных в виде pickle файла
    try:
        with open(file_path, 'wb') as file:
            pickle.dump(table, file)
    except Exception as e:
        raise e
        
def save_text(table, file_path):    #сохранение таблицы в текстовом файле
        try:
            with open(file_path, 'w') as file:
                for row in table.data:
                    file.write(','.join(map(str, row)) + '\n')
        except Exception as e:
            raise e
            
#создаем модуль с базовыми операциями над таблицами
class Table:
    def __init__(self, data):
        self.data = data

    def get_rows_by_number(self, start, stop=None, copy_table=False):   #извлечение строк из таблицы по номеру строки
        if stop is None:
            stop = start + 1
        if copy_table:
            return Table(self.data[start:stop])
        else:
            self.data = self.data[start:stop]

    def get_rows_by_index(self, *args, copy_table=False):           #извлечение строк из таблицы по значениям из первого столбца
        indices = [i for i, row in enumerate(self.data) if row[0] in args]
        if copy_table:
            return Table([self.data[i] for i in indices])
        else:
            self.data = [self.data[i] for i in indices]

    def get_column_types(self, by_number=True):                #получение типов столбцов
        types_dict = {}
        for i in range(len(self.data[0])):
            if by_number:
                types_dict[i] = type(self.data[0][i]).__name__
            else:
                types_dict[self.data[0][i]] = type(self.data[0][i]).__name__
        return types_dict

    def set_column_types(self, types_dict, by_number=True):     #задание типов столбцов
        for i in range(len(self.data[0])):
            if by_number:
                self.data[0][i] = types_dict.get(i)(self.data[0][i])
            else:
                self.data[0][i] = types_dict.get(self.data[0][i])(self.data[0][i])

    def get_values(self, column=0):                        #получение значений таблицы из столбца
        try:
            column_values = [row[column] for row in self.data]
            return [type(column_values[0])(value) for value in column_values]
        except IndexError:
            raise IndexError("Column index out of range")

    def get_value(self, column=0):                         #получение значения таблицы с одной строкой
        try:
            return type(self.data[0][column])(self.data[0][column])
        except IndexError:
            raise IndexError("Column index out of range")

    def set_values(self, values, column=0):    #задание списка значений  для столбца
        try:
            for i in range(len(self.data)):
                self.data[i][column] = type(values[0])(values[i])
        except IndexError:
            raise IndexError("Column index out of range")

    def set_value(self, value, column=0):  #задание значения для таблицы с одной строкой
        try:
            self.data[0][column] = type(value)(value)
        except IndexError:
            raise IndexError("Column index out of range")

    def print_table(self):             #вывод таблицы
        for row in self.data:
            print(row)